# Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set
import warnings
import re
from pandas.io import gbq
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy import sparse

# Get the Data

In [2]:
chicago_data = """
                    SELECT unique_key, date, primary_type, location_description, 
                            arrest, domestic, community_area,year, latitude, longitude,
                    FROM `gdac-327115.Chicago.chicago2`
                    WHERE year >= 2011
               """
chicago_data = gbq.read_gbq(chicago_data, project_id="gdac-327115")

In [3]:
#Read in an Excel file with a one to one mapping between Chicago community areas and districts
chicago_districts = pd.read_excel("ChicagoCommunityAreas.xlsx")

In [4]:
#Data type can't be joined on an int
chicago_districts.community_area = chicago_districts["community_area"].astype("string")
chicago_data.community_area = chicago_data["community_area"].astype("string")

In [5]:
#Outer join the two data sets
chicago = chicago_data.merge(chicago_districts, how="outer", left_on="community_area", right_on="community_area", )

In [6]:
#Check data types
chicago.dtypes

unique_key                            Int64
date                    datetime64[ns, UTC]
primary_type                         object
location_description                 object
arrest                              boolean
domestic                            boolean
community_area                       string
year                                  Int64
latitude                            float64
longitude                           float64
district_name                        object
community_name                       object
dtype: object

In [7]:
#Change the object data types to categorical
chicago.loc[:, chicago.dtypes == "object"] = chicago.select_dtypes(["object"]).apply(lambda x: x.astype("category"))

In [8]:
#Drop the community area variable since we have a community name variable
chicago.drop("community_area", axis = 1, inplace = True)

### Explore first few rows

In [10]:
chicago.head()

,unique_key,date,primary_type,location_description,arrest,domestic,year,latitude,longitude,district_name,community_name
0,11228094,2017-11-20 15:00:00+00:00,DECEPTIVE PRACTICE,POOL ROOM,False,False,2017,NaN,NaN,Far North,NORTH PARK
1,12135292,2020-08-13 12:00:00+00:00,DECEPTIVE PRACTICE,BANK,False,False,2020,41.993402,-87.712280,Far North,NORTH PARK
2,8905294,2012-11-27 17:30:00+00:00,PUBLIC PEACE VIOLATION,RESTAURANT,False,False,2012,41.978265,-87.708742,Far North,NORTH PARK
3,8805120,2012-09-17 00:30:00+00:00,DECEPTIVE PRACTICE,TAXICAB,False,False,2012,41.985277,-87.713827,Far North,NORTH PARK
4,9030552,2013-02-21 00:00:00+00:00,CRIM SEXUAL ASSAULT,RESIDENCE,False,True,2013,41.994323,-87.720213,Far North,NORTH PARK


In [11]:
#Check the number of rows and columns
chicago.shape

(2971810, 11)

# Split the Data into a Training Set and Test Set

In [35]:
chicago_train = chicago.loc[chicago["year"] != 2021]
chicago_test = chicago.loc[chicago["year"] == 2021]

In [20]:
#Check if the test set contains data from the full year
chicago_test.loc[:, "date"].dt.month. \
                                value_counts(). \
                                reset_index(). \
                                rename(columns={"index":"Month", "date":"Count"}). \
                                sort_values(by = "Month")

,Month,Count
5,1,16008
8,2,12852
6,3,15709
7,4,15279
4,5,17493
2,6,18468
0,7,18898
3,8,18110
1,9,18632
9,10,5507


# Data Exploration and Cleaning

In [22]:
#Check the total number of missing values for each variable
chicago_train.isnull().sum()

unique_key                  0
date                        0
primary_type                0
location_description     7844
arrest                      0
domestic                    0
year                        0
latitude                26843
longitude               26843
district_name             218
community_name            218
dtype: int64

In [21]:
#Check to make sure there are no duplicate incidents based on the "unique key" variable
assert len(chicago_train["unique_key"].unique()) == chicago_train.shape[0]

### Cleaning Primary Type Variable

In [23]:
#Quick reference on the number of missing values
chicago_train["primary_type"].isnull().sum()

0

In [10]:
#Examine the different crime types listed
chicago_train["primary_type"].value_counts()

THEFT                                636007
BATTERY                              512318
CRIMINAL DAMAGE                      299943
NARCOTICS                            222406
ASSAULT                              189549
OTHER OFFENSE                        171368
DECEPTIVE PRACTICE                   167709
BURGLARY                             152532
MOTOR VEHICLE THEFT                  120013
ROBBERY                              108092
CRIMINAL TRESPASS                     69971
WEAPONS VIOLATION                     45871
OFFENSE INVOLVING CHILDREN            23359
PUBLIC PEACE VIOLATION                21829
PROSTITUTION                          12438
CRIM SEXUAL ASSAULT                   12334
INTERFERENCE WITH PUBLIC OFFICER      11793
SEX OFFENSE                           10771
HOMICIDE                               5669
ARSON                                  4482
GAMBLING                               3507
LIQUOR LAW VIOLATION                   3407
CRIMINAL SEXUAL ASSAULT         

It appears "CRIM SEXUAL ASSAULT" should be merged into "CRIMINAL SEXUAL ASSAULT". We'll also merge the three various "Non-criminal" categories into one category. We'll do the same thing for the "OTHER NARCOTIC VIOLATION" category. Finally, we'll move "RITUALISM" into the "OTHER" since it has only one occurence. Let's collect all of these changes into one function. 

In [11]:
def crime_type_cleaner(df):
    """
    This function cleans the primary_type column in place. 
    
    df: DataFrame with primary_type column
    
    returns: None
    """
    df.loc[:, "primary_type"].replace(["CRIM SEXUAL ASSAULT"], "CRIMINAL SEXUAL ASSAULT", inplace = True)

    df.loc[:, "primary_type"].replace(["OTHER NARCOTIC VIOLATION"], "NARCOTICS", inplace = True)

    df.loc[:, "primary_type"].replace(["NON - CRIMINAL", "NON-CRIMINAL (SUBJECT SPECIFIED)"], "NON-CRIMINAL", inplace = True)

    df.loc[:, "primary_type"].replace(["RITUALISM"], "OTHER OFFENSE", inplace = True)
   

In [29]:
#Clean the primary type variable
crime_type_cleaner(chicago_train)
#Check the cleaned variable
chicago_train["primary_type"].value_counts()

THEFT                                636007
BATTERY                              512318
CRIMINAL DAMAGE                      299943
NARCOTICS                            222467
ASSAULT                              189549
OTHER OFFENSE                        171369
DECEPTIVE PRACTICE                   167709
BURGLARY                             152532
MOTOR VEHICLE THEFT                  120013
ROBBERY                              108092
CRIMINAL TRESPASS                     69971
WEAPONS VIOLATION                     45871
OFFENSE INVOLVING CHILDREN            23359
PUBLIC PEACE VIOLATION                21829
CRIMINAL SEXUAL ASSAULT               14850
PROSTITUTION                          12438
INTERFERENCE WITH PUBLIC OFFICER      11793
SEX OFFENSE                           10771
HOMICIDE                               5669
ARSON                                  4482
GAMBLING                               3507
LIQUOR LAW VIOLATION                   3407
KIDNAPPING                      

### Cleaning Location Description Variable

In [13]:
#Check the number of missing values
chicago_train["location_description"].isnull().sum()

7844

In [14]:
#Look at which rows are missing
chicago_train.loc[chicago_train["location_description"].isnull()]

,unique_key,date,primary_type,location_description,arrest,domestic,year,latitude,longitude,district_name,community_name
192,10625982,2015-08-01 16:20:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2015,NaN,NaN,Far North,NORTH PARK
285,12185129,2020-09-28 21:37:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2020,41.987639,-87.710214,Far North,NORTH PARK
2019,11489624,2018-10-26 15:05:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2018,41.994360,-87.723939,Far North,NORTH PARK
2034,12018775,2020-03-26 14:00:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2020,41.974920,-87.706215,Far North,NORTH PARK
2042,10596491,2016-06-16 13:00:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2016,41.985369,-87.712605,Far North,NORTH PARK
...,...,...,...,...,...,...,...,...,...,...,...
2970607,10465221,2016-03-24 14:25:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2016,41.982365,-87.662518,Far North,EDGEWATER
2970615,12114559,2020-07-23 11:58:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2020,41.991336,-87.656851,Far North,EDGEWATER
2970618,10883083,2017-03-19 11:55:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2017,NaN,NaN,Far North,EDGEWATER
2970632,10852091,2017-02-16 17:15:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2017,41.982755,-87.662528,Far North,EDGEWATER


Looks like there might be some relationship between the missing locations and the type of crime committed. Let's further explore this. 

In [36]:
#Look at the counts of the primary type variable with the data filtered to those values of location that are missing
chicago_train.loc[chicago_train["location_description"].isnull()]["primary_type"].value_counts()

DECEPTIVE PRACTICE                   7124
THEFT                                 704
BURGLARY                                5
ROBBERY                                 4
BATTERY                                 2
CRIMINAL DAMAGE                         2
OTHER OFFENSE                           1
ARSON                                   1
CRIMINAL SEXUAL ASSAULT                 1
SEX OFFENSE                             0
NON-CRIMINAL                            0
RITUALISM                               0
PUBLIC PEACE VIOLATION                  0
PUBLIC INDECENCY                        0
PROSTITUTION                            0
STALKING                                0
OTHER NARCOTIC VIOLATION                0
OFFENSE INVOLVING CHILDREN              0
OBSCENITY                               0
NON-CRIMINAL (SUBJECT SPECIFIED)        0
MOTOR VEHICLE THEFT                     0
NON - CRIMINAL                          0
NARCOTICS                               0
ASSAULT                           

It appears that nearly all missing values for location description have a primary type that is labeled as "DECEPTIVE PRACTICE". Let's see what the non-missing values for location description have for their corresponding "DECEPTIVE PRACTICE" values.

In [24]:
#Create a boolean mask that filters the data to non-missing locations with a primary type of "DECEPTIVE PRACTICE"
mask = (chicago_train["location_description"].notnull()) & (chicago_train["primary_type"] == "DECEPTIVE PRACTICE")
chicago_train.loc[mask]["location_description"].value_counts()

RESIDENCE                54361
APARTMENT                17232
OTHER                    16680
STREET                    8369
BANK                      6910
                         ...  
PORCH                        0
POOLROOM                     0
EXPRESSWAY EMBANKMENT        0
FARM                         0
YMCA                         0
Name: location_description, Length: 197, dtype: int64

Thus, it appears that the "RESIDENCE" location occurrs the most for non-missing "DECEPTIVE PRACTICE" crimes. Therefore, we can impute the missing values for location that have the corresponding crime as "DECEPTIVE PRACTICE" with "RESIDENCE".  

Let's do the same process for the "THEFT" crime

In [23]:
#Create a boolean mask that filters the data to non-missing locations with a primary type of "THEFT"
mask = (chicago_train["location_description"].notnull()) & (chicago_train["primary_type"] == "THEFT")
chicago_train.loc[mask]["location_description"].value_counts()

STREET                152284
RESIDENCE              58720
APARTMENT              41793
SMALL RETAIL STORE     35004
DEPARTMENT STORE       32695
                       ...  
ELEVATOR                   0
DRIVEWAY                   0
SCHOOL YARD                0
CTA PROPERTY               0
YMCA                       0
Name: location_description, Length: 197, dtype: int64

Now, it appears that the "STREET" location is the most occurring for non-missing "THEFT" crimes. Let's impute the missing values for location that have the corresponding crime as "THEFT" with "STREET". 

Finally, we'll collect the remaining crimes into one group due to their low counts and check what crime is most occurring for the group.

In [37]:
crime_type = ["BURGLARY", "ROBBERY", "BATTERY", "CRIMINAL DAMAGE", "ARSON", "CRIMINAL SEXUAL ASSAULT", "OTHER OFFENSE"]
mask = (chicago_train["primary_type"].isin(crime_type)) & (chicago_train["location_description"].notnull())
chicago_train.loc[mask, ["location_description"]].value_counts()

location_description
RESIDENCE               277616
APARTMENT               248008
STREET                  235210
SIDEWALK                125040
RESIDENCE-GARAGE         35372
                         ...  
PARKING LOT                  0
CLUB                         0
OFFICE                       0
NURSING HOME                 0
YMCA                         0
Length: 197, dtype: int64

Again, it looks like "RESIDENCE" is the most occuring location for the group of crimes when location is not missing. Thus, we can impute the missing location values for the group of crimes as "RESIDENCE". 

We'll collect all of these missing value imputations into one function

In [38]:
def location_imputer(df):
    """
    This function imputes missing values in the location_description column by setting the missing value to the value that 
    occurs most in the primary_type column in place
    
    df: DataFrame that contains the location_description column
    
    returns: None
    """
    mask = (df.loc[:, "primary_type"] == "DECEPTIVE PRACTICE") & (df.loc[:, "location_description"].isnull())
    df.loc[mask, ["location_description"]] = "RESIDENCE"
    
    mask = (df.loc[:, "primary_type"] == "THEFT") & (df.loc[:, "location_description"].isnull())
    df.loc[mask, ["location_description"]] = "STREET"
    
    crime_type = ["BURGLARY", "ROBBERY", "BATTERY", "CRIMINAL DAMAGE", "ARSON", "CRIMINAL SEXUAL ASSAULT", "OTHER OFFENSE"]
    mask = (df.loc[:, "primary_type"].isin(crime_type)) & (df.loc[:, "location_description"].isnull())
    df.loc[mask, ["location_description"]] = "RESIDENCE"
    

In [40]:
#Impute the missing values
location_imputer(chicago_train)
#Check that there are no missing values
chicago_train["location_description"].isnull().sum()

0

Now we can examine the different locations

In [53]:
#Take a look at the different categories in the location description variable
pd.set_option("display.max_rows", None)
chicago_train["location_description"].value_counts()

STREET                                                   638800
RESIDENCE                                                473271
APARTMENT                                                366599
SIDEWALK                                                 277648
OTHER                                                    101780
PARKING LOT/GARAGE(NON.RESID.)                            76267
SMALL RETAIL STORE                                        60930
ALLEY                                                     59350
RESTAURANT                                                55421
RESIDENTIAL YARD (FRONT/BACK)                             53164
VEHICLE NON-COMMERCIAL                                    48626
RESIDENCE PORCH/HALLWAY                                   46118
RESIDENCE-GARAGE                                          44805
SCHOOL, PUBLIC, BUILDING                                  43233
DEPARTMENT STORE                                          41196
GAS STATION                             

Clearly, there are too many categories, 197, to be used in a model with most variables having very few occurrences. Let's see if we can't combine some of the similar values into one category. For instance, there are numerous different ways to break down Chicago Transit Authority (CTA) which can all be combined into one CTA category

In [51]:
chicago_train.loc[chicago_train["location_description"].str.contains("CTA")]["location_description"].value_counts()[:20]

CTA TRAIN                                    15906
CTA PLATFORM                                 12793
CTA BUS                                      11066
CTA BUS STOP                                  6331
CTA STATION                                   5767
CTA GARAGE / OTHER PROPERTY                   4863
CTA TRACKS - RIGHT OF WAY                      168
CTA PARKING LOT / GARAGE / OTHER PROPERTY      137
CTA "L" PLATFORM                                 6
CTA PROPERTY                                     3
CTA SUBWAY STATION                               1
CTA "L" TRAIN                                    1
POLICE FACILITY/VEH PARKING LOT                  0
POLICE FACILITY / VEHICLE PARKING LOT            0
PAWN SHOP                                        0
PARKING LOT/GARAGE(NON.RESID.)                   0
PARKING LOT / GARAGE (NON RESIDENTIAL)           0
ABANDONED BUILDING                               0
HOTEL/MOTEL                                      0
POOL ROOM                      

We'll do this process of combining categories by putting them all into one function

In [54]:
def location_cleaner(df):
    """
    This function cleans the location_description by merging categories together that are logically related in place.
    
    df: DataFrame that contains the location_description column
    
    returns: None
    """

    #Combine all values related to CTA
    CTA = ["CTA TRAIN", "CTA PLATFORM", "CTA BUS", "CTA BUS STOP", "CTA STATION", "CTA GARAGE / OTHER PROPERTY", 
           "CTA TRACKS - RIGHT OF WAY", "CTA PARKING LOT / GARAGE / OTHER PROPERTY", """CTA "L" PLATFORM""",
          "CTA PROPERTY", """CTA "L" TRAIN""", "CTA SUBWAY STATION"]
    df["location_description"].replace(CTA, "CTA", inplace = True)

    #Combine GANGWAY into ALLEY
    df["location_description"].replace(["GANGWAY"], "ALLEY", inplace = True)

    #Combine all values related to Sports/Clubs
    sports = ["ATHLETIC CLUB", "SPORTS ARENA/STADIUM", "SPORTS ARENA / STADIUM", "CLUB"]
    df["location_description"].replace(sports, "SPORTS", inplace = True)

    #Combine all values related to VACANT
    vacant = ["VACANT LOT/LAND", "VACANT LOT / LAND", "VACANT LOT"]
    df["location_description"].replace(vacant, "VACANT", inplace = True)

    #Combine the two Gas Station values
    df["location_description"].replace(["GAS STATION DRIVE/PROP."], "GAS STATION", inplace = True)

    #Combine all values related to Water/River/Lake
    water = ["LAKEFRONT/WATERFRONT/RIVERBANK", "POOL ROOM", "BOAT/WATERCRAFT", "LAKEFRONT / WATERFRONT / RIVERBANK", 
             "BOAT / WATERCRAFT", "RIVER BANK", "POOLROOM", "LAKE", "LAGOON"]
    df["location_description"].replace(water, "WATER", inplace = True)

    #Combine all values related to Banks/Credit Unions
    bank = ["ATM (AUTOMATIC TELLER MACHINE)", "CURRENCY EXCHANGE", "CREDIT UNION", "SAVINGS AND LOAN"]
    df["location_description"].replace(bank, "BANK", inplace = True)

    #Combine all values related to Roads/Streets/Highway
    street = ["HIGHWAY/EXPRESSWAY", "BRIDGE", "HIGHWAY / EXPRESSWAY", "EXPRESSWAY EMBANKMENT"]
    df["location_description"].replace(street, "STREET", inplace = True)

    #Combine all values related to Residence/Houses or items inside a House
    residence = ["RESIDENCE PORCH/HALLWAY", "RESIDENCE-GARAGE", "RESIDENCE - YARD (FRONT / BACK)", "RESIDENCE - PORCH / HALLWAY", 
           "RESIDENCE - GARAGE", "RESIDENTIAL YARD (FRONT/BACK)", "HOUSE", "ROOMING HOUSE", "PORCH", "YARD", "HALLWAY", "GARAGE",
          "VESTIBULE", "STAIRWELL", "BASEMENT", "LAUNDRY ROOM", "DRIVEWAY", "DRIVEWAY - RESIDENTIAL"]
    df["location_description"].replace(residence, "RESIDENCE", inplace = True)

    #Combine all values related to College
    college = ["COLLEGE/UNIVERSITY GROUNDS", "COLLEGE/UNIVERSITY RESIDENCE HALL", "COLLEGE / UNIVERSITY - GROUNDS", 
           "COLLEGE / UNIVERSITY - RESIDENCE HALL"]
    df["location_description"].replace(college, "COLLEGE", inplace = True)

    #Combine all values related to CTA into one CTA category
    veh = ["VEHICLE NON-COMMERCIAL", "VEHICLE-COMMERCIAL", "VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)", 
           "VEHICLE - OTHER RIDE SERVICE", "VEHICLE - DELIVERY TRUCK", "VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)", 
           "VEHICLE - COMMERCIAL", "VEHICLE-COMMERCIAL - TROLLEY BUS", "VEHICLE-COMMERCIAL - ENTERTAINMENT/PARTY BUS", 
           "VEHICLE - COMMERCIAL: TROLLEY BUS", "VEHICLE - COMMERCIAL: ENTERTAINMENT / PARTY BUS", "TAXICAB", "",
          "AUTO / BOAT / RV DEALERSHIP", "AUTO", "DELIVERY TRUCK", "GARAGE/AUTO REPAIR", "TAXI CAB", "TRUCK", "TRAILER"]
    df["location_description"].replace(veh, "VEHICLE", inplace = True)

    #Combine all values related to Police/Fire Station/Protection
    police = ["POLICE FACILITY/VEH PARKING LOT", "POLICE FACILITY / VEHICLE PARKING LOT", "JAIL / LOCK-UP FACILITY",
             "FIRE STATION", "FOREST PRESERVE"]
    df["location_description"].replace(police, "POLICE", inplace = True)

    #Combine all values related to Parking
    parking = ["PARKING LOT/GARAGE(NON.RESID.)", "CHA PARKING LOT/GROUNDS", "PARKING LOT / GARAGE (NON RESIDENTIAL)", 
            "CHA PARKING LOT / GROUNDS", "PARKING LOT", "CHA PARKING LOT"]
    df["location_description"].replace(parking, "PARKING", inplace = True)

    #Combine all values related to Airports
    air = ["AIRPORT TERMINAL UPPER LEVEL - SECURE AREA", "AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA", 
           "AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA", "AIRPORT VENDING ESTABLISHMENT", "AIRPORT/AIRCRAFT", "AIRCRAFT", 
           "AIRPORT PARKING LOT", "AIRPORT EXTERIOR - NON-SECURE AREA", "AIRPORT TERMINAL LOWER LEVEL - SECURE AREA", 
           "AIRPORT BUILDING NON-TERMINAL - SECURE AREA", "AIRPORT EXTERIOR - SECURE AREA", "AIRPORT TRANSPORTATION SYSTEM (ATS)",
           "AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA", "AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA"]
    df["location_description"].replace(air, "AIRPORT", inplace = True)

    #Combine all values related to School/Day Care
    school = ["SCHOOL, PUBLIC, BUILDING", "SCHOOL, PUBLIC, GROUNDS", "SCHOOL - PUBLIC BUILDING", "SCHOOL - PUBLIC GROUNDS", 
              "PUBLIC HIGH SCHOOL", "SCHOOL, PRIVATE, BUILDING", "SCHOOL, PRIVATE, GROUNDS", "SCHOOL - PRIVATE BUILDING", 
              "SCHOOL - PRIVATE GROUNDS", "SCHOOL YARD", "DAY CARE CENTER"]
    df["location_description"].replace(school, "SCHOOL", inplace = True)

    #Combine all values related to Bar/Tavern/Restaurant
    restaurant = ["BAR OR TAVERN", "TAVERN"]
    df["location_description"].replace(restaurant, "RESTAURANT", inplace = True)

    #Combine all values related to stores
    store = ["SMALL RETAIL STORE", "DEPARTMENT STORE", "GROCERY FOOD STORE", "CONVENIENCE STORE", "DRUG STORE", 
             "TAVERN/LIQUOR STORE", "CLEANING STORE", "APPLIANCE STORE", "TAVERN / LIQUOR STORE", "RETAIL STORE", 
             "LIQUOR STORE", "NEWSSTAND", "PAWN SHOP", "MOVIE HOUSE/THEATER", "MOVIE HOUSE / THEATER", "BARBERSHOP", 
             "CAR WASH", "COIN OPERATED MACHINE", "BOWLING ALLEY","KENNEL", "BARBER SHOP/BEAUTY SALON", "CLEANERS/LAUNDROMAT"]
    df["location_description"].replace(store, "STORE", inplace = True)

    #Combine all values related to hospitals or nursing homes
    hos = ["HOSPITAL BUILDING/GROUNDS", "HOSPITAL BUILDING / GROUNDS", "ANIMAL HOSPITAL", "NURSING HOME/RETIREMENT HOME", 
           "NURSING / RETIREMENT HOME", "NURSING HOME"]
    df["location_description"].replace(hos, "HOSPITAL", inplace = True)

    #Combine all values related to hotels/motels
    hotel = ["HOTEL/MOTEL", "HOTEL / MOTEL", "MOTEL"]
    df["location_description"].replace(hotel, "HOTEL", inplace = True)

    #Combine all values related to offices
    office = ["COMMERCIAL / BUSINESS OFFICE", "MEDICAL/DENTAL OFFICE", "MEDICAL / DENTAL OFFICE"]
    df["location_description"].replace(office, "OFFICE", inplace = True)

    #Combine all values related to buildings
    building = ["ABANDONED BUILDING", "GOVERNMENT BUILDING/PROPERTY", "FACTORY/MANUFACTURING BUILDING", 
                "GOVERNMENT BUILDING / PROPERTY", "FEDERAL BUILDING", "FACTORY / MANUFACTURING BUILDING", "GOVERNMENT BUILDING",
               "WAREHOUSE", "ELEVATOR", "YMCA"]
    df["location_description"].replace(building, "BUILDING", inplace = True)

    #Combine all values related to Chicago Housing Authority (CHA)
    cha = ["CHA APARTMENT", "CHA HALLWAY/STAIRWELL/ELEVATOR", "CHA HALLWAY / STAIRWELL / ELEVATOR", "CHA GROUNDS", 
           "CHA PLAY LOT", "CHA HALLWAY", "CHA ELEVATOR"]
    df["location_description"].replace(cha, "CHA", inplace = True)

    #Combine all values related to Church
    church = ["CHURCH/SYNAGOGUE/PLACE OF WORSHIP", "CHURCH / SYNAGOGUE / PLACE OF WORSHIP", "CHURCH PROPERTY"]
    df["location_description"].replace(church, "CHURCH", inplace = True)

    #Combine all other miscellaneous vaules
    other = ["OTHER (SPECIFY)", "OTHER RAILROAD PROP / TRAIN DEPOT", "OTHER COMMERCIAL TRANSPORTATION", 
             "OTHER RAILROAD PROPERTY / TRAIN DEPOT", "CEMETARY", "FARM", "HORSE STABLE", "RAILROAD PROPERTY",
             "WOODED AREA", "SEWER"]
    df["location_description"].replace(other, "OTHER", inplace = True)
    


In [56]:
#Clean location description variable
location_cleaner(chicago_train)
chicago_train["location_description"].value_counts()

STREET               639494
RESIDENCE            634663
APARTMENT            366599
SIDEWALK             277648
STORE                174996
OTHER                110876
PARKING               88353
RESTAURANT            74485
SCHOOL                64492
ALLEY                 59376
VEHICLE               57290
CTA                   57042
GAS STATION           33424
PARK PROPERTY         21477
BANK                  20019
OFFICE                19074
HOSPITAL              18387
BUILDING              16748
HOTEL                 13006
AIRPORT               12855
VACANT                11034
POLICE                10114
CHA                    9771
SPORTS                 7598
CHURCH                 6033
CONSTRUCTION SITE      3427
LIBRARY                2511
COLLEGE                2469
WATER                  1593
Name: location_description, dtype: int64

We reduced the number of categories from 197 to only 29. 

In [59]:
#Reset display options
pd.options.display.max_rows = 15